In [ ]:
#Standard Header used on the projects

#first the major packages used for math and graphing
import numpy as np
import matplotlib.pyplot as plt
from cycler import cycler
import scipy.special as sp
#import pandas as pd

#Custome graph format style sheet
#plt.style.use('Prospectus.mplstyle')

#If being run by a seperate file, use the seperate file's graph format and saving paramaeters
#otherwise set what is needed
if not 'Saving' in locals():
    Saving = True
if not 'Titles' in locals():
    Titles = True
if not 'Ledgends' in locals():
    Ledgends = True
if not 'FFormat' in locals():
    FFormat = '.png'
Computer = "SciClone"   # "SciClone"    "WinLap"  "LinLap"   "Desktop"  "PortLap"

#Standard cycle for collors and line styles
default_cycler = (cycler('color', ['0.00', '0.40', '0.60', '0.70']) + cycler(linestyle=['-', '--', ':', '-.']))
plt.rc('axes', prop_cycle=default_cycler)
my_cmap = plt.get_cmap('gray')

In [ ]:
import datetime
import multiprocessing
from joblib import Parallel, delayed
import pywt
import os as os
import statistics as st
import os as os
import random
import time
import platform
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import cv2
from sklearn.model_selection import train_test_split


tf.config.set_visible_devices([], 'GPU')


In [4]:
HostName = platform.node()
dataSize = 'big'

if HostName == "Server":
    Computer = "Desktop"   
elif HostName[-6:] == 'wm.edu':
    Computer = "SciClone"
elif HostName == "SchoolLaptop":
    Computer = "LinLap"
elif HostName == "WTC-TAB-512":
    Computer = "PortLap"
else:
    Computer = "WinLap"

if Computer == "SciClone":
    location = '/sciclone/home20/dchendrickson01/'
elif Computer == "WinLap":
    location = 'C:\\Data\\'
elif Computer == "Desktop":
    location = "E:\\Backups\\Dan\\CraneData\\"
elif Computer == "LinLap":
    location = '/home/dan/Output/'

if Computer ==  "SciClone":
    rootfolder = '/sciclone/home20/dchendrickson01/'
    if dataSize == 'big':
        folder = '/sciclone/scr10/dchendrickson01/CraneData/'
        imFolder ='/sciclone/scr10/dchendrickson01/BigPrints/'
    else:
        folder = '/sciclone/data10/dchendrickson01/SmallCopy/'
elif Computer == "Desktop":
    rootfolder = location
    if dataSize == 'big':
        folder = 'G:\\CraneData\\'
    else:
        folder = rootfolder + "SmallCopy\\"
elif Computer =="WinLap":
    rootfolder = location
    folder = rootfolder + "SmallCopy\\"   
elif Computer == "LinLap":
    rootfolder = '/home/dan/Data/'
    folder = rootfolder + 'SmallCopy/'
    
#Pre Calculations
num_cores = multiprocessing.cpu_count()-1


num_classes = 2
input_shape = (500, 10000, 3)

In [5]:
                                 
Good_files =[os.path.join(dp, f) for dp, dn, fn in os.walk(os.path.expanduser(imFolder+'Good/')) for f in fn]
Bad_files =[os.path.join(dp, f) for dp, dn, fn in os.walk(os.path.expanduser(imFolder+'Bad/')) for f in fn]

print('Have Files')

Have Files


In [7]:
Images = []
Truth = []
  
def addImage(file):
    pathname, extension = os.path.splitext(file)
    filetree = pathname.split('/')
    foldername = filetree[-2]
    if foldername == 'Good':
        truth = [0]
        image = cv2.imread(file, flags=cv2.IMREAD_COLOR)
        return truth,image
    elif foldername == 'Bad':
        truth = [1]
        image = cv2.imread(file, flags=cv2.IMREAD_COLOR)
        return truth,image
    else:
        return None
j=0

tg = Parallel(n_jobs=num_cores)(delayed(addImage)(file) for file in Good_files[:1000])
tb = Parallel(n_jobs=num_cores)(delayed(addImage)(file) for file in Bad_files[:1000])


Truth = np.concatenate((tg[0], tb[0]), axis=0)
Images = np.concatenate((tg[1], tb[1]), axis=0)

for file in Good_files[:1000]:
    t ,i  = addImage(file) 
    Images.append(np.asarray(i))
    Truth.append(np.asarray(t))
    j+=1
    if j%1000==0:print(j)
j=0
for file in Bad_files[:1000]:
    t ,i  = addImage(file) 
    Images.append(np.asarray(i))
    Truth.append(np.asarray(t))
    j+=1
    if j%1000==0:print(j)
print('sorted')

In [8]:
Images=np.asarray(Images)
Truth=np.asarray(Truth)

In [9]:

x_train, x_test,y_train,y_test = train_test_split(Images,Truth, test_size=0.15, shuffle=True, random_state=0)



In [10]:
print(f"x_train shape: {np.shape(x_train)} - y_train shape: {np.shape(y_train)}")
print(f"x_test shape: {np.shape(x_test)} - y_test shape: {np.shape(y_test)}")

"""
## Configure the hyperparameters
"""

x_train shape: (3,) - y_train shape: (3,)
x_test shape: (1,) - y_test shape: (1,)


'\n## Configure the hyperparameters\n'

In [11]:
learning_rate = 0.001
weight_decay = 0.0001
batch_size = 256
num_epochs = 100
image_l = 10000  # We'll resize input images to this size
image_w = 500
patch_size = 10  # Size of the patches to be extract from the input images
num_patches = (image_l // patch_size) * (image_w // patch_size)
projection_dim = 64
num_heads = 4
transformer_units = [
    projection_dim * 2,
    projection_dim,
]  # Size of the transformer layers
transformer_layers = 8
mlp_head_units = [2048, 1024]  # Size of the dense layers of the final classifier

In [12]:
"""
## Use data augmentation


data_augmentation = keras.Sequential(
    [
        layers.Normalization(),
        layers.Resizing(image_size, image_size),
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(factor=0.02),
        layers.RandomZoom(height_factor=0.2, width_factor=0.2),
    ],
    name="data_augmentation",
)
# Compute the mean and the variance of the training data for normalization.
data_augmentation.layers[0].adapt(x_train)



## Implement multilayer perceptron (MLP)
"""


def mlp(x, hidden_units, dropout_rate):
    for units in hidden_units:
        x = layers.Dense(units, activation=tf.nn.gelu)(x)
        x = layers.Dropout(dropout_rate)(x)
    return x


"""
## Implement patch creation as a layer
"""


class Patches(layers.Layer):
    def __init__(self, patch_size):
        super(Patches, self).__init__()
        self.patch_size = patch_size

    def call(self, images):
        batch_size = tf.shape(images)[0]
        patches = tf.image.extract_patches(
            images=images,
            sizes=[1, self.patch_size, self.patch_size, 1],
            strides=[1, self.patch_size, self.patch_size, 1],
            rates=[1, 1, 1, 1],
            padding="VALID",
        )
        patch_dims = patches.shape[-1]
        patches = tf.reshape(patches, [batch_size, -1, patch_dims])
        patches=np.asarray(patches)
        return patches

In [15]:
    
"""
Let's display patches for a sample image
"""

import matplotlib.pyplot as plt

plt.figure(figsize=(10, 5))
image = x_train[np.random.choice(range(x_train.shape[0]))]
plt.imshow(image.astype("uint8"))
plt.axis("off")

resized_image = [image] #tf.image.resize(
#    tf.convert_to_tensor([image]), size=(image_size, image_size)
#)

patches = Patches(patch_size)(resized_image)
print(f"Image size: {image_l} X {image_w}")
print(f"Patch size: {patch_size} X {patch_size}")
print(f"Patches per image: {patches.shape[1]}")
print(f"Elements per patch: {patches.shape[-1]}")

AttributeError: 'list' object has no attribute 'astype'

<Figure size 720x360 with 0 Axes>

n = int(image_l / patch_size) #int(np.sqrt(patches.shape[1]))
m = int(image_w / patch_size)
plt.figure(figsize=(4, 4))
for i, patch in enumerate(patches[0]):
    ax = plt.subplot(n, m, i + 1)
    patch_img = tf.reshape(patch, (patch_size, patch_size, 3))
    plt.imshow(patch_img.numpy().astype("uint8"))
    plt.axis("off")

In [14]:
"""
## Implement the patch encoding layer
The `PatchEncoder` layer will linearly transform a patch by projecting it into a
vector of size `projection_dim`. In addition, it adds a learnable position
embedding to the projected vector.
"""

'\n## Implement the patch encoding layer\nThe `PatchEncoder` layer will linearly transform a patch by projecting it into a\nvector of size `projection_dim`. In addition, it adds a learnable position\nembedding to the projected vector.\n'

In [16]:
class PatchEncoder(layers.Layer):
    def __init__(self, num_patches, projection_dim):
        super(PatchEncoder, self).__init__()
        self.num_patches = num_patches
        self.projection = layers.Dense(units=projection_dim)
        self.position_embedding = layers.Embedding(
            input_dim=num_patches, output_dim=projection_dim
        )

    def call(self, patch):
        positions = tf.range(start=0, limit=self.num_patches, delta=1)
        encoded = self.projection(patch) + self.position_embedding(positions)
        return encoded


"""
## Build the ViT model
The ViT model consists of multiple Transformer blocks,
which use the `layers.MultiHeadAttention` layer as a self-attention mechanism
applied to the sequence of patches. The Transformer blocks produce a
`[batch_size, num_patches, projection_dim]` tensor, which is processed via an
classifier head with softmax to produce the final class probabilities output.
Unlike the technique described in the [paper](https://arxiv.org/abs/2010.11929),
which prepends a learnable embedding to the sequence of encoded patches to serve
as the image representation, all the outputs of the final Transformer block are
reshaped with `layers.Flatten()` and used as the image
representation input to the classifier head.
Note that the `layers.GlobalAveragePooling1D` layer
could also be used instead to aggregate the outputs of the Transformer block,
especially when the number of patches and the projection dimensions are large.
"""


def create_vit_classifier():
    inputs = layers.Input(shape=(image_w,image_l))
    # Augment data.
    augmented = inputs #data_augmentation(inputs)
    # Create patches.
    patches = Patches(patch_size)(augmented)
    # Encode patches.
    encoded_patches = PatchEncoder(num_patches, projection_dim)(patches)

    # Create multiple layers of the Transformer block.
    for _ in range(transformer_layers):
        # Layer normalization 1.
        x1 = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
        # Create a multi-head attention layer.
        attention_output = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=projection_dim, dropout=0.1
        )(x1, x1)
        # Skip connection 1.
        x2 = layers.Add()([attention_output, encoded_patches])
        # Layer normalization 2.
        x3 = layers.LayerNormalization(epsilon=1e-6)(x2)
        # MLP.
        x3 = mlp(x3, hidden_units=transformer_units, dropout_rate=0.1)
        # Skip connection 2.
        encoded_patches = layers.Add()([x3, x2])

    # Create a [batch_size, projection_dim] tensor.
    representation = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
    representation = layers.Flatten()(representation)
    representation = layers.Dropout(0.5)(representation)
    # Add MLP.
    features = mlp(representation, hidden_units=mlp_head_units, dropout_rate=0.5)
    # Classify outputs.
    logits = layers.Dense(num_classes)(features)
    # Create the Keras model.
    model = keras.Model(inputs=inputs, outputs=logits)
    return model


"""
## Compile, train, and evaluate the mode
"""

def run_experiment(model):
    optimizer = tf.keras.optimizers.SGD(learning_rate = learning_rate,
                                momentum=0.0, 
                                nesterov=False, 
                                name='SGD')

    model.compile(
        optimizer=optimizer,
        loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=[
            keras.metrics.SparseCategoricalAccuracy(name="accuracy"),
            keras.metrics.SparseTopKCategoricalAccuracy(5, name="top-5-accuracy"),
        ],
    )

    checkpoint_filepath = "/tmp/checkpoint"
    checkpoint_callback = keras.callbacks.ModelCheckpoint(
        checkpoint_filepath,
        monitor="val_accuracy",
        save_best_only=True,
        save_weights_only=True,
    )

    history = model.fit(
        x=x_train,
        y=y_train,
        batch_size=batch_size,
        epochs=num_epochs,
        validation_split=0.1,
        callbacks=[checkpoint_callback],
    )

    model.load_weights(checkpoint_filepath)
    _, accuracy, top_5_accuracy = model.evaluate(x_test, y_test)
    print(f"Test accuracy: {round(accuracy * 100, 2)}%")
    print(f"Test top 5 accuracy: {round(top_5_accuracy * 100, 2)}%")

    return history

In [17]:
vit_classifier = create_vit_classifier()
history = run_experiment(vit_classifier)

ValueError: Exception encountered when calling layer "patches" (type Patches).

in user code:

    File "/local/scr/dchendrickson01/TMPDIR/ipykernel_175450/3543310222.py", line 43, in call  *
        patches = tf.image.extract_patches(

    ValueError: Shape must be rank 4 but is rank 3 for '{{node patches/ExtractImagePatches}} = ExtractImagePatches[T=DT_FLOAT, ksizes=[1, 10, 10, 1], padding="VALID", rates=[1, 1, 1, 1], strides=[1, 10, 10, 1]](Placeholder)' with input shapes: [?,500,10000].


Call arguments received:
  • images=tf.Tensor(shape=(None, 500, 10000), dtype=float32)

In [ ]:
model.save(imFolder+'Model/')

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.savefig(imFolder + 'ModelAccuracy.png')
plt.show()

# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.savefig(imFolder + 'ModelLoss.png')
plt.show()